In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category=UserWarning)

env: TOKENIZERS_PARALLELISM=false


In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import torch
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/TP3_LLM/100_enron_spam_data.csv')
df

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,Date
0,0,6568,kirstee ' s role in london,vince :\nthis is precisely the concern i have ...,ham,2000-07-25
1,1,9797,i never sent you this - keep it hush hush ! : ...,online credit breakthroughrepair your credit o...,spam,2002-08-02
2,2,655,copanos changes,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,2000-04-12
3,3,8206,"hrgovcic , hrvoje",please incease the bonus for hrgovcic in vince...,ham,2001-01-13
4,4,25787,a chance to get new logo now,working on your company ' s image ? start with...,spam,2005-06-27
...,...,...,...,...,...,...
195,195,22338,not for sale in the u . a . e - usb voip hands...,not for sale in the u . a . e\nusb voip handse...,spam,2005-07-27
196,196,6329,re : improve communication,"zimin ,\ni will make sure that you have the op...",ham,2000-06-22
197,197,33146,giga medzz,dear sir / madam .\nwe are pleased to introduc...,spam,2005-06-11
198,198,24384,improved health aegjngkjgw,go\nbig . . . . . . supersize today ! ! !\n* *...,spam,2002-07-05


In [ ]:
df = df[:110]

In [ ]:
generation_args = {
    "max_new_tokens": 350,
    "return_full_text": False,
    "temperature": 0.3,
    "do_sample": True,
}

In [ ]:
import pprint
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message
    text_template = """{} Class of the email:?""".format(row["Message"])

    # Set up the messages for the assistant
    messages = [
        {"role": "system", "content": "Classify the text into Spam or Ham. Please provide me at the end of the response with Class of the email:. "},
        {"role": "user", "content": text_template},
    ]

    # Call your pipe function (replace `pipe` and `generation_args` with actual implementation)
    output = pipe(messages, **generation_args)

    # Extract the generated text
    generated_text = output[0]["generated_text"]
    print("example {} : \n".format(index))
    pprint.pprint(generated_text)
    print("\n")

    # Extract the class of the email
    if "Class of the email:" in generated_text:
        # Extract the portion of the text after "Class of the email:"
        email_class = generated_text.split("Class of the email:")[-1].strip().split()[0]  # Take the first word

    pprint.pprint(email_class)
    print("\n")
    #In our prompt if we write it spam or ham in the response it didn't give the class so we used instead "Spam" or "Ham" to recognize it and we formatted it to make it like the true labels
    if email_class == "Spam" or email_class == "Spam." :
        email_class = "spam"
    elif email_class == "Ham":
        email_class = "ham"

    # Print the actual Spam/Ham label from the DataFrame for comparison
    actual_class = row["Spam/Ham"]
    pprint.pprint(actual_class)


    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


example 0 : 

' Class of the email: Ham'


'Ham'


'ham'
example 1 : 

(' Class of the email: Spam\n'
 '\n'
 'The email contains phrases such as "online credit breakthrough", "clear up '
 'bad credit online", and "watch your credit daily" which are common in spam '
 'emails. The email also includes a link with a suspicious domain name (tm) '
 'and a request for immediate action, which are also indicators of spam.')


'Spam'


'spam'
example 2 : 

' Class of the email: Ham'


'Ham'


'ham'
example 3 : 

' Class of the email: Spam'


'Spam'


'ham'
example 4 : 

' Class of the email: Spam'


'Spam'


'spam'
example 5 : 

(' Class of the email: Spam\n'
 '\n'
 'The email contains promotional content about discounted ink cartridges, free '
 'shipping, and various brand mentions. It also includes a call to action to '
 'opt-out of the advertisement and a link to a website. These characteristics '
 'are typical of spam emails.')


'Spam'


'spam'
example 6 : 

(' Class of the email: Spam\n'
 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


example 9 : 

' Class of the email: Ham'


'Ham'


'spam'
example 10 : 

' Class of the email: Spam'


'Spam'


'spam'
example 11 : 

' Class of the email: Ham'


'Ham'


'ham'
example 12 : 

(' Class of the email: Spam\n'
 '\n'
 'The email is likely spam because it uses a generic salutation ("dear '
 'valued"), includes a link to a non-specific "paypal ® security center," and '
 'pressures the recipient to update their account records to avoid suspension. '
 "It also lacks personalization and specific details about the recipient's "
 'account.')


'Spam'


'spam'
example 13 : 

' Class of the email: Ham'


'Ham'


'ham'
example 14 : 

' Class of the email: Ham'


'Ham'


'ham'
example 15 : 

' Class of the email: Spam'


'Spam'


'spam'
example 16 : 

' Class of the email: Ham'


'Ham'


'ham'
example 17 : 

' Class of the email: Ham'


'Ham'


'spam'
example 18 : 

' Class of the email: Spam'


'Spam'


'spam'
example 19 : 

(' Class of the email: Spam\n'
 '\n'
 'The email contains p

In [ ]:
set(predicted_labels)

{'Business/Work-related', 'ham', 'spam'}

In [ ]:
label_mapping = {'ham': 0, 'spam': 1}
# Define valid labels
valid_labels = set(label_mapping.keys())  # e.g., {'spam', 'ham'}

# Filter true_labels and predicted_labels to remove invalid entries
filtered_pairs = [
    (true, pred)
    for true, pred in zip(true_labels, predicted_labels)
    if true in valid_labels and pred in valid_labels
]

# Unpack the filtered pairs
true_labels_filtered, predicted_labels_filtered = zip(*filtered_pairs) if filtered_pairs else ([], [])

# Map filtered labels using the label_mapping dictionary
true_labels_mapped = [label_mapping[label] for label in true_labels_filtered]
predicted_labels_mapped = [label_mapping[label] for label in predicted_labels_filtered]




In [ ]:
len(predicted_labels_mapped)

109

In [ ]:
len(true_labels_mapped)

109

In [ ]:
true_labels_mapped

[0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0]

In [ ]:
predicted_labels = ["spam" if label == "Spam." else label for label in predicted_labels]

# Print the updated list
print(predicted_labels)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Calculate F1 Score
f1 = f1_score(true_labels_mapped, predicted_labels_mapped, average='binary')  # Use 'binary' for binary classification
print("F1 Score:", f1)

# Calculate Accuracy
accuracy = accuracy_score(true_labels_mapped, predicted_labels_mapped)
print("Accuracy:", accuracy)

# Calculate Precision
precision = precision_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Precision:", precision)

# Calculate Recall
recall = recall_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Recall:", recall)


F1 Score: 0.8545454545454545
Accuracy: 0.8532110091743119
Precision: 0.8103448275862069
Recall: 0.9038461538461539


In [ ]:

# Print the metrics
metrics = {
    "F1 Score": f1,
    "Precision": precision,
    "Recall": recall,
    "Accuracy": accuracy
}

pprint.pprint(metrics)
print("\n")